In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import Data & Manipulate Data

In [25]:
df_org = pd.read_csv("[2010-2020]XAUUSD-M5-NoSession.csv")

## Set DateTime as Index

In [26]:
df_org["DateTime"] = pd.to_datetime(df_org["DateTime"])
df_org = df_org.set_index("DateTime")
df_org = df_org.sort_index()
df_org.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 715180 entries, 2010-01-01 00:00:00 to 2020-01-01 23:55:00
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Open    715180 non-null  float64
 1   High    715180 non-null  float64
 2   Low     715180 non-null  float64
 3   Close   715180 non-null  float64
 4   Volume  715180 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 32.7 MB


# Select Columns

In [73]:
df = df_org[["Open", "High", "Low", "Close"]]
df

,Open,High,Low,Close
DateTime,,,,
2010-01-01 00:00:00,1096.09,1096.21,1096.02,1096.07
2010-01-01 00:05:00,1096.15,1096.17,1095.99,1096.04
2010-01-01 00:10:00,1096.06,1096.15,1095.99,1096.15
2010-01-01 00:15:00,1096.07,1096.17,1096.05,1096.17
2010-01-01 00:20:00,1096.08,1096.20,1096.05,1096.15
...,...,...,...,...
2020-01-01 23:35:00,1519.30,1519.52,1518.77,1519.28
2020-01-01 23:40:00,1519.29,1519.61,1519.11,1519.33
2020-01-01 23:45:00,1519.33,1519.33,1518.41,1518.59


# Feature Engineering

## Adding base Ichimoku-KinkoHyo

#### Normal means : current time frame
#### Advanced mean : multi time looking at current timeframe

### Writing Function for better calculation

In [75]:
def add_ichimoku(df, multi_time=3):
    tk, kj, s_b = 9, 26, 52
    for i in range(multi_time):
        i += 1
        """           Normal Parameters             """
        
        """      Calculating Normal Tenkan-Sen      """
        high_for_tk = df["High"].rolling(window=tk*i, min_periods=tk*i).max()
        low_for_tk = df["Low"].rolling(window=tk*i, min_periods=tk*i).min()
        df[f"Tenkan_sen_norm_{i}X"] = (high_for_tk + low_for_tk) / 2
        
        """     Calculating Normal Kijun-Sen        """
        high_for_kj = df["High"].rolling(window=kj*i, min_periods=kj*i).max()
        low_for_kj = df["Low"].rolling(window=kj*i, min_periods=kj*i).min()
        df[f"Kijun_sen_norm_{i}X"] = (high_for_kj + low_for_kj) / 2
        
        """     Calculating Normal Senkou-span-A    """
        df[f"Senkou_span_A_norm_{i}X"] = (df[f"Tenkan_sen_norm_{i}X"] + df[f"Kijun_sen_norm_{i}X"]) / 2
        
        """     Calculating Normal Senkou-span-B    """
        high_for_s_b = df["High"].rolling(window=s_b*i, min_periods=s_b*i).max()
        low_for_s_b = df["Low"].rolling(window=s_b*i, min_periods=s_b*i).min()
        df[f"Senkou_span_B_norm_{i}X"] = (high_for_s_b + low_for_s_b) / 2
        
        """           Advanced Parameters            """
        """      Calculating Avanced Tenkan-Sen      """
        df[f"Tenkan_sen_adv_{i}X"] = df[f"Tenkan_sen_norm_{i}X"].shift(kj*i)
        
        """      Calculating Avanced Kijun-Sen       """
        df[f"Kijun_sen_adv_{i}X"] = df[f"Kijun_sen_norm_{i}X"].shift(kj*i)
        
        """     Calculating Avanced Senkou-span-A    """
        df[f"Senkou_span_A_adv_{i}X"] = df[f"Senkou_span_A_norm_{i}X"].shift(kj*i)
        
        """     Calculating Avanced Senkou-span-B    """
        df[f"Senkou_span_B_adv_{i}X"] = df[f"Senkou_span_B_norm_{i}X"].shift(kj*i)
        
        """      Calculating Avanced Chikou-span      """
        df[f"Chikou_span_adv_{i}X"] = df["Close"].shift(kj*i)
        
    return df

In [76]:
add_ichimoku(df, multi_time=4)
df

,Open,High,Low,Close,Kijun_sen_norm_1X,Tenkan_sen_norm_1X,Senkou_span_A_norm_1X,Senkou_span_B_norm_1X,Tenkan_sen_adv_1X,Kijun_sen_adv_1X,...,Chikou_span_adv_3X,Tenkan_sen_norm_4X,Kijun_sen_norm_4X,Senkou_span_A_norm_4X,Senkou_span_B_norm_4X,Tenkan_sen_adv_4X,Kijun_sen_adv_4X,Senkou_span_A_adv_4X,Senkou_span_B_adv_4X,Chikou_span_adv_4X
DateTime,,,,,,,,,,,,,,,,,,,,,
2010-01-01 00:00:00,1096.09,1096.21,1096.02,1096.07,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-01 00:05:00,1096.15,1096.17,1095.99,1096.04,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-01 00:10:00,1096.06,1096.15,1095.99,1096.15,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-01 00:15:00,1096.07,1096.17,1096.05,1096.17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-01 00:20:00,1096.08,1096.20,1096.05,1096.15,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-01 23:35:00,1519.30,1519.52,1518.77,1519.28,1518.875,1518.875,1518.8750,1518.955,1519.300,1519.680,...,1521.20,1518.875,1520.66,1519.7675,1520.875,1521.975,1522.515,1522.245,1519.825,1520.74
2020-01-01 23:40:00,1519.29,1519.61,1519.11,1519.33,1518.875,1519.920,1519.3975,1518.875,1519.300,1519.485,...,1520.33,1518.875,1520.66,1519.7675,1520.875,1521.975,1522.515,1522.245,1519.825,1521.98
2020-01-01 23:45:00,1519.33,1519.33,1518.41,1518.59,1518.875,1519.740,1519.3075,1518.875,1519.225,1519.415,...,1520.86,1518.875,1520.66,1519.7675,1520.875,1521.975,1522.515,1522.245,1519.825,1522.21


# Calculate TK & KJ - S_A & S_B Positions

### No. 0 For Down Trend
### No. 1 For Up Trend

In [ ]:
def Tk_Kj_position(df):
    

# Shift All Columns values for better performance

In [15]:
columnss = list(df.columns)
columnss = columnss[4:]
for col_name in columnss:
    df[col_name] = df[col_name].shift(1)
df

,Open,High,Low,Close,Tk_norm,Kj_norm,S_A_norm,S_B_norm,Tk_adv,Kj_adv,...,S_B_adv_x3,Chiko_adv_x3,Tk_norm_x4,S_A_norm_x4,S_B_norm_x4,Tk_adv_x4,Kj_adv_x4,S_A_adv_x4,S_B_adv_x4,Chiko_adv_x4
DateTime,,,,,,,,,,,,,,,,,,,,,
2010-01-01 00:00:00,1096.09,1096.21,1096.02,1096.07,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-01 00:05:00,1096.15,1096.17,1095.99,1096.04,1096.115,1096.115,1096.1150,1096.115,NaN,NaN,...,NaN,NaN,1096.115,1096.1150,1096.115,NaN,NaN,NaN,NaN,NaN
2010-01-01 00:10:00,1096.06,1096.15,1095.99,1096.15,1096.100,1096.100,1096.1000,1096.100,NaN,NaN,...,NaN,NaN,1096.100,1096.1000,1096.100,NaN,NaN,NaN,NaN,NaN
2010-01-01 00:15:00,1096.07,1096.17,1096.05,1096.17,1096.100,1096.100,1096.1000,1096.100,NaN,NaN,...,NaN,NaN,1096.100,1096.1000,1096.100,NaN,NaN,NaN,NaN,NaN
2010-01-01 00:20:00,1096.08,1096.20,1096.05,1096.15,1096.100,1096.100,1096.1000,1096.100,NaN,NaN,...,NaN,NaN,1096.100,1096.1000,1096.100,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-01 23:35:00,1519.30,1519.52,1518.77,1519.28,1518.875,1518.875,1518.8750,1519.315,1519.310,1520.040,...,1520.875,1521.70,1518.875,1519.7675,1520.875,1521.975,1522.515,1522.245,1519.825,1521.13
2020-01-01 23:40:00,1519.29,1519.61,1519.11,1519.33,1518.875,1518.875,1518.8750,1518.955,1519.300,1519.680,...,1520.875,1521.20,1518.875,1519.7675,1520.875,1521.975,1522.515,1522.245,1519.825,1520.74
2020-01-01 23:45:00,1519.33,1519.33,1518.41,1518.59,1519.920,1518.875,1519.3975,1518.875,1519.300,1519.485,...,1520.875,1520.33,1518.875,1519.7675,1520.875,1521.975,1522.515,1522.245,1519.825,1521.98


### Drop Nan Rows

In [78]:
df.dropna(inplace=True)

# Nerual Network simulation


!(1_-teDpAIho_nzNShRswkfrQ.gif)